In [ ]:
!pip install gym-2048

     |████████████████████████████████| 1.5MB 3.5MB/s 
     |████████████████████████████████| 13.8MB 288kB/s 
  Created wheel for gym-2048: filename=gym_2048-0.2.6-cp37-none-any.whl size=4699 sha256=dd1f843760d4eaa537d14419927c8f7c8b06857224b45510c22b321b830616e6
  Stored in directory: /root/.cache/pip/wheels/ab/11/22/a6c0fb3622f6fad7e8b7f4342a3c64e115bee08a4189bbae18
  Created wheel for gym: filename=gym-0.10.11-cp37-none-any.whl size=1588312 sha256=3cbd0119f5c433fb8d80ed4facaea004fc260949387b184dd75d22a30cdc73da
  Stored in directory: /root/.cache/pip/wheels/7b/eb/1f/22c4124f3c64943aa0646daf4612b1c1f00f27d89b81304ebd
Successfully built gym-2048 gym
ERROR: xarray 0.18.2 has requirement numpy>=1.17, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tifffile 2021.4.8 has requirement numpy>=1.15.1, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.14.6 which is incompatible.
ERROR: spacy 2.2.4 has 

In [ ]:
import gym
import gym_2048
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Custom class to be able to define the inital starting state - going to be used for Monte Carlo tree search.

In [ ]:
class NewBase2048Env(gym_2048.Base2048Env):
  def __init__(self, *args):
    gym_2048.Base2048Env.__init__(self, *args)
  def custom_reset(self, starting_board): #new function to allow reset with initial conditions
    self.board = starting_board
    return self.board

# Monte Carlo Tree Search Algorithm

This function takes two arguments:

- `run_simulations`: The number of move sets the algorithm should simulate before making each real move
- `move_simulations`: The number of moves it should simulate in each run

For instance, if `run_simulations` is 50 and `move_simulations` is 10, then before each real move, the algorithm will simulate 50 random sets of 10 moves. Increasing these values will improve performance but increase the time to compute as well.

The algorithm will track the total score accumulated during each simulated run, and will choose the next move to be the one which had the highest average score for all of its simulated runs. For instance, in the example above, the algorithm will test 50 simulated runs of 10 random moves. For each possible move (up, down, left, right), the algorithm will find the average total score accumulated for all random runs which began with that move. Then, it will choose the next move to be the one which had the highest average score accumulated.

In [ ]:
import time

def monte_carlo_2048(run_simulations=50, move_simulations=10, criterion='avg'):
  start = time.time()
  env = NewBase2048Env()

  obs, reward, done, _ = env.step(2)
  moves_made = 0
  total_score = 0
  while not done:
    current_board = env.board.copy() #making a copy of the current board

    future_moves = {0:[], 1:[], 2:[], 3:[]} #list of all ending scores for each possible next move
    for simulated_run in range(run_simulations):
      reward = 0

      simulated_env = NewBase2048Env()
      simulated_env.custom_reset(current_board) #initiate a simulated environment with the current board setup

      action = np.random.choice([0, 1, 2, 3]) #first action
      _, score, lost, _ = simulated_env.step(action)
      reward += score #add the number of points from the move to the total reward from this run
      if lost: #if the game is done, set reward to 0 and break the simulated run
        reward = 0
        
      for simulated_move in range(move_simulations): #additional random actions
        _, score, lost, _ = simulated_env.step(np.random.choice([0, 1, 2, 3]))
        reward += score
        if lost:
          break
          
      future_moves[action].append(reward) #append the total points accumulated from this run to the future_moves dictionary
    
    if criterion=='max':
      #look at max rewards for each possible next move, and choose the one with highest max reward
      best_reward = 0
      best_move = 0
      for key in future_moves:
        if np.max(future_moves[key]) > best_reward:
          best_reward = np.max(future_moves[key])
          best_move = key
    else:
      #look at average rewards for each possible next move, and choose the one with highest average
      best_reward = 0
      best_move = 0
      for key in future_moves:
        if np.mean(future_moves[key]) > best_reward:
          best_reward = np.mean(future_moves[key])
          best_move = key
    
    obs, reward, done, _ = env.step(best_move)
    total_score += reward
    moves_made += 1
    if moves_made % 50 == 0:
      print(f'{moves_made} moves made')
      print(f'Total Score: {total_score}')
      print(f'Current best tile: {np.max(env.board)}')
      print()

      
  print(f'Final Score: {total_score}')
  print(f'Time to train: {time.time()-start}')
  print(f'Best tile achieved: {np.max(env.board)}')
  print()
  env.render()

  return time.time()-start, np.max(env.board), total_score

In [ ]:
monte_carlo_2048(run_simulations=100, move_simulations=10)

50 moves made
Total Score: 444
Current best tile: 64

100 moves made
Total Score: 1088
Current best tile: 128

150 moves made
Total Score: 1968
Current best tile: 256

200 moves made
Total Score: 2620
Current best tile: 256

250 moves made
Total Score: 3548
Current best tile: 256

300 moves made
Total Score: 4536
Current best tile: 512

350 moves made
Total Score: 5248
Current best tile: 512

400 moves made
Total Score: 6152
Current best tile: 512

450 moves made
Total Score: 6744
Current best tile: 512

500 moves made
Total Score: 9296
Current best tile: 1024

550 moves made
Total Score: 9956
Current best tile: 1024

600 moves made
Total Score: 10908
Current best tile: 1024

650 moves made
Total Score: 11560
Current best tile: 1024

700 moves made
Total Score: 12032
Current best tile: 1024

750 moves made
Total Score: 13484
Current best tile: 1024

800 moves made
Total Score: 14164
Current best tile: 1024

850 moves made
Total Score: 15104
Current best tile: 1024

900 moves made
Total

(1119.4169249534607, 2048, 35828)

Running a fixed number of simulations for each possible next move instead of totally random moves **SLIGHT MODIFICATION OF ABOVE**

In [ ]:
import time

def monte_carlo_2048(run_simulations=50, move_simulations=10, criterion='avg'):
  start = time.time()
  env = NewBase2048Env()

  obs, reward, done, _ = env.step(2)
  moves_made = 0
  total_score = 0
  while not done:
    current_board = env.board.copy() #making a copy of the current board

    future_moves = {0:[], 1:[], 2:[], 3:[]} #list of all ending scores for each possible next move
    for action in [0, 1, 2, 3]:
      for simulated_run in range(run_simulations):
        reward = 0

        simulated_env = NewBase2048Env()
        simulated_env.custom_reset(current_board) #initiate a simulated environment with the current board setup

        _, score, lost, _ = simulated_env.step(action)
        reward += score #add the number of points from the move to the total reward from this run
        if lost: #if the game is done, set reward to 0 and break the simulated run
          reward = 0
          
        for simulated_move in range(move_simulations): #additional random actions
          _, score, lost, _ = simulated_env.step(np.random.choice([0, 1, 2, 3]))
          reward += score
          if lost:
            break
            
        future_moves[action].append(reward) #append the total points accumulated from this run to the future_moves dictionary
    
    if criterion=='max':
      #look at max rewards for each possible next move, and choose the one with highest max reward
      best_reward = 0
      best_move = 0
      for key in future_moves:
        if np.max(future_moves[key]) > best_reward:
          best_reward = np.max(future_moves[key])
          best_move = key
    else:
      #look at average rewards for each possible next move, and choose the one with highest average
      best_reward = 0
      best_move = 0
      for key in future_moves:
        if np.mean(future_moves[key]) > best_reward:
          best_reward = np.mean(future_moves[key])
          best_move = key
    
    obs, reward, done, _ = env.step(best_move)
    total_score += reward
    moves_made += 1
    if moves_made % 50 == 0:
      print(f'{moves_made} moves made')
      print(f'Total Score: {total_score}')
      print(f'Current best tile: {np.max(env.board)}')
      print()

      
  print(f'Final Score: {total_score}')
  print(f'Time to train: {time.time()-start}')
  print(f'Best tile achieved: {np.max(env.board)}')
  print()
  env.render()

  return time.time()-start, np.max(env.board), total_score

In [ ]:
monte_carlo_2048()

50 moves made
Total Score: 448
Current best tile: 64

100 moves made
Total Score: 1116
Current best tile: 128

150 moves made
Total Score: 1968
Current best tile: 256

200 moves made
Total Score: 2672
Current best tile: 256

250 moves made
Total Score: 4128
Current best tile: 512

300 moves made
Total Score: 4796
Current best tile: 512

350 moves made
Total Score: 5292
Current best tile: 512

400 moves made
Total Score: 6168
Current best tile: 512

450 moves made
Total Score: 6884
Current best tile: 512

500 moves made
Total Score: 9368
Current best tile: 1024

550 moves made
Total Score: 9964
Current best tile: 1024

600 moves made
Total Score: 10908
Current best tile: 1024

650 moves made
Total Score: 11572
Current best tile: 1024

700 moves made
Total Score: 12072
Current best tile: 1024

750 moves made
Total Score: 13496
Current best tile: 1024

800 moves made
Total Score: 14196
Current best tile: 1024

850 moves made
Total Score: 15112
Current best tile: 1024

900 moves made
Total

(1744.9487600326538, 2048, 27256)